## Практическое задание

<ol>
    <li>Попробуйте обучить нейронную сеть на TensorFlow 2 на любом датасете imdb_reviews. 
        Опишите в комментарии к уроку - какой результата вы добились от нейросети? Что помогло вам улучшить ее точность?<br><br>
    </li>
    <li>*2. Поработайте с документацией TensorFlow 2. Найдите полезные команды не разобранные на уроке.</li>
    
    
</ol>

In [2]:
!pip install tensorflow_hub
!pip install tensorflow_datasets

  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21499 sha256=e34816a8cc7bdc02093972316010cad239d57068ef7bfc544e67122d54bf7461
  Stored in directory: c:\users\kashe\appdata\local\pip\cache\wheels\29\93\c6\762e359f8cb6a5b69c72235d798804cae523bbe41c2aa8333d
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78597 sha256=1cee4c368e252665451692181c74ba836415753727ccd188650f6425d460af2e
  Stored in directory: c:\users\kashe\appdata\local\pip\cache\wheels\a4\61\fd\c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for googleapis-common-protos: filename=googleapis_common_protos-1.51.0-py3-none-any.whl size=77602 sha256=1a9a0072a797c132a8bbd612d02847b0dfd59d976ace5605568a55efc8fa5970
  Stored in directory: c:\users\kashe\appdata\local\pip\cache\wheels\4c\a1\71\5e427276ceeff277fd76878d1b19fbf4587a2845015d86864b
Successfully built promise dill googleapis-common-protos


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [4]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.1.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


### Скачиваем Dataset

In [20]:
# Разобьем обучающую выборку в пропорции 60% на 40%, и у нас будет 15,000 примеров
# для обучения, 10,000 примеров для валидации и 25,000 примеров для проверки.
(train_data, validation_data, raw_test), test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:30%]','train[30%:50%]', 'train[50%:]'),
    with_info=True,
    as_supervised=True)

In [21]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'This is a big step down after the surprisingly enjoyable original. This sequel isn\'t nearly as fun as part one, and it instead spends too much time on plot development. Tim Thomerson is still the best thing about this series, but his wisecracking is toned down in this entry. The performances are all adequate, but this time the script lets us down. The action is merely routine and the plot is only mildly interesting, so I need lots of silly laughs in order to stay entertained during a "Trancers" movie. Unfortunately, the laughs are few and far between, and so, this film is watchable at best.',
       b"Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven't seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on me now. However, I will say that it left an impression on me... and most of my friends

In [22]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 1, 0, 1, 0, 1, 1, 1, 0], dtype=int64)>

### Построение модели

In [24]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 2.209591  , -2.7093675 ,  3.6802928 , -1.0291991 , -4.1671185 ,
        -2.4566064 , -2.2519937 , -0.36589956,  1.9485804 , -3.1104462 ,
        -2.4610963 ,  1.3139242 , -0.9161584 , -0.16625322, -3.723651  ,
         1.8498232 ,  3.499562  , -1.2373022 , -2.8403084 , -1.213074  ],
       [ 1.9055302 , -4.11395   ,  3.6038654 ,  0.28555924, -4.658998  ,
        -5.5433393 , -3.2735848 ,  1.9235417 ,  3.8461034 ,  1.5882455 ,
        -2.64167   ,  0.76057523, -0.14820506,  0.9115291 , -6.45758   ,
         2.3990374 ,  5.0985413 , -3.2776263 , -3.2652326 , -1.2345369 ],
       [ 3.6510668 , -4.7066135 ,  4.71003   , -1.7002777 , -3.7708545 ,
        -3.709126  , -4.222776  ,  1.946586  ,  6.1182513 , -2.7392752 ,
        -5.4384456 ,  2.7078724 , -2.1263676 , -0.7084146 , -5.893995  ,
         3.1602864 ,  3.8389287 , -3.318196  , -5.1542974 , -2.4051712 ]],
      dtype=float32)>

In [25]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
15/15 [==============================] - 3s 214ms/step - loss: 0.8829 - accuracy: 0.5004 - val_loss: 0.8101 - val_accuracy: 0.4606
Epoch 2/20
15/15 [==============================] - 2s 146ms/step - loss: 0.7687 - accuracy: 0.4883 - val_loss: 0.7497 - val_accuracy: 0.5110
Epoch 3/20
15/15 [==============================] - 2s 157ms/step - loss: 0.7187 - accuracy: 0.5489 - val_loss: 0.7134 - val_accuracy: 0.5518
Epoch 4/20
15/15 [==============================] - 2s 147ms/step - loss: 0.6797 - accuracy: 0.5896 - val_loss: 0.6809 - val_accuracy: 0.5882
Epoch 5/20
15/15 [==============================] - 2s 158ms/step - loss: 0.6487 - accuracy: 0.6296 - val_loss: 0.6495 - val_accuracy: 0.6356
Epoch 6/20
15/15 [==============================] - 3s 168ms/step - loss: 0.6172 - accuracy: 0.6669 - val_loss: 0.6189 - val_accuracy: 0.6658
Epoch 7/20
15/15 [==============================] - 2s 157ms/step - loss: 0.5880 - accuracy: 0.7015 - val_loss: 0.5930 - val_accuracy: 0.6960
Epoch 

In [28]:
results = model.evaluate(raw_test.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.378
accuracy: 0.841


In [29]:
history = model.fit(train_data.shuffle(10000).batch(1024),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
8/8 [==============================] - 3s 339ms/step - loss: 0.2937 - accuracy: 0.8973 - val_loss: 0.3717 - val_accuracy: 0.8458
Epoch 2/20
8/8 [==============================] - 2s 283ms/step - loss: 0.2838 - accuracy: 0.9027 - val_loss: 0.3676 - val_accuracy: 0.8446
Epoch 3/20
8/8 [==============================] - 3s 318ms/step - loss: 0.2754 - accuracy: 0.9064 - val_loss: 0.3620 - val_accuracy: 0.8490
Epoch 4/20
8/8 [==============================] - 3s 361ms/step - loss: 0.2664 - accuracy: 0.9093 - val_loss: 0.3579 - val_accuracy: 0.8484
Epoch 5/20
8/8 [==============================] - 2s 308ms/step - loss: 0.2572 - accuracy: 0.9115 - val_loss: 0.3541 - val_accuracy: 0.8492
Epoch 6/20
8/8 [==============================] - 3s 313ms/step - loss: 0.2503 - accuracy: 0.9179 - val_loss: 0.3496 - val_accuracy: 0.8538
Epoch 7/20
8/8 [==============================] - 2s 297ms/step - loss: 0.2426 - accuracy: 0.9204 - val_loss: 0.3467 - val_accuracy: 0.8512
Epoch 8/20
8/8 [====

In [30]:
results = model.evaluate(raw_test.batch(1024), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.317
accuracy: 0.866


In [31]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 128), dtype=float32, numpy=
array([[ 1.1329565 , -0.03431067,  0.12162483,  0.23909318, -0.00590096,
         0.01708133,  0.08015746, -0.24174148,  0.02650847, -0.04987909,
         0.11247347, -0.5601333 , -0.19981343, -0.33171475, -0.21482827,
        -0.0408943 , -0.07008895,  0.04761497, -0.4271519 ,  0.5916435 ,
         0.20662342, -0.01224876,  0.02093347,  0.08682311,  0.1036322 ,
        -0.27562743,  0.08033531, -0.1508889 , -0.21847422,  0.06831451,
        -0.14723632, -0.01117621,  0.25246456,  0.10763772,  0.16948967,
        -0.0666896 , -0.2209269 , -0.3792632 , -0.15618715,  0.20775793,
        -0.24780062,  0.05174338,  0.04574278, -0.03784208,  0.09963844,
         0.2984763 ,  0.04106063, -0.1306813 , -0.23743384,  0.07203843,
         0.09253472, -0.13241911, -0.29098544,  0.1412372 , -0.20057504,
        -0.02892039, -0.14496696, -0.11467611,  0.35817885,  0.137189  ,
        -0.0982758 ,  0.04868827,  0.01760143, -0.08307212, -0.0034984 ,
 

In [32]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 124,644,769
Trainable params: 124,644,769
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [34]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
15/15 [==============================] - 29s 2s/step - loss: 0.6654 - accuracy: 0.6272 - val_loss: 0.6214 - val_accuracy: 0.7430
Epoch 2/20
15/15 [==============================] - 27s 2s/step - loss: 0.5625 - accuracy: 0.8025 - val_loss: 0.5432 - val_accuracy: 0.7830
Epoch 3/20
15/15 [==============================] - 28s 2s/step - loss: 0.4595 - accuracy: 0.8519 - val_loss: 0.4644 - val_accuracy: 0.8216
Epoch 4/20
15/15 [==============================] - 27s 2s/step - loss: 0.3507 - accuracy: 0.9005 - val_loss: 0.3969 - val_accuracy: 0.8490
Epoch 5/20
15/15 [==============================] - 28s 2s/step - loss: 0.2577 - accuracy: 0.9373 - val_loss: 0.3537 - val_accuracy: 0.8616
Epoch 6/20
15/15 [==============================] - 27s 2s/step - loss: 0.1823 - accuracy: 0.9591 - val_loss: 0.3285 - val_accuracy: 0.8660
Epoch 7/20
15/15 [==============================] - 27s 2s/step - loss: 0.1191 - accuracy: 0.9753 - val_loss: 0.3241 - val_accuracy: 0.8626
Epoch 8/20
15/15 [==

In [36]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=6,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/6
15/15 [==============================] - 30s 2s/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.4222 - val_accuracy: 0.8584
Epoch 2/6
15/15 [==============================] - 27s 2s/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.4279 - val_accuracy: 0.8582
Epoch 3/6
15/15 [==============================] - 27s 2s/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.4323 - val_accuracy: 0.8582
Epoch 4/6
15/15 [==============================] - 27s 2s/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.4380 - val_accuracy: 0.8588
Epoch 5/6
15/15 [==============================] - 29s 2s/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.4411 - val_accuracy: 0.8588
Epoch 6/6
15/15 [==============================] - 32s 2s/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.4455 - val_accuracy: 0.8594


In [37]:
results = model.evaluate(raw_test.batch(1024), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.449
accuracy: 0.854


In [38]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])
model = tf.keras.Sequential()

model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary() 

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 9         
Total params: 400,501
Trainable params: 400,501
Non-trainable params: 0
_________________________________________________________________


In [43]:
history = model.fit(train_data.shuffle(10000).batch(1024),
                    epochs=10,
                    validation_data=validation_data.batch(1024),
                    verbose=1)

Epoch 1/10
8/8 [==============================] - 2s 289ms/step - loss: 0.1938 - accuracy: 0.9340 - val_loss: 0.3582 - val_accuracy: 0.8496
Epoch 2/10
8/8 [==============================] - 2s 270ms/step - loss: 0.1875 - accuracy: 0.9359 - val_loss: 0.3556 - val_accuracy: 0.8536
Epoch 3/10
8/8 [==============================] - 2s 285ms/step - loss: 0.1733 - accuracy: 0.9437 - val_loss: 0.3590 - val_accuracy: 0.8516
Epoch 4/10
8/8 [==============================] - 2s 284ms/step - loss: 0.1641 - accuracy: 0.9467 - val_loss: 0.3549 - val_accuracy: 0.8516
Epoch 5/10
8/8 [==============================] - 2s 286ms/step - loss: 0.1521 - accuracy: 0.9516 - val_loss: 0.3535 - val_accuracy: 0.8534
Epoch 6/10
8/8 [==============================] - 2s 273ms/step - loss: 0.1425 - accuracy: 0.9567 - val_loss: 0.3524 - val_accuracy: 0.8546
Epoch 7/10
8/8 [==============================] - 2s 279ms/step - loss: 0.1332 - accuracy: 0.9624 - val_loss: 0.3533 - val_accuracy: 0.8560
Epoch 8/10
8/8 [====

In [44]:
results = model.evaluate(raw_test.batch(1024), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.350
accuracy: 0.858


In [45]:
# Разобьем обучающую выборку в другой пропорции
(train_data, validation_data, raw_test), test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:50%]','train[50%:80%]', 'train[80%:]'),
    with_info=True,
    as_supervised=True)

In [46]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])
model = tf.keras.Sequential()

model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary() 

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_7 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [47]:
history = model.fit(train_data.shuffle(10000).batch(1024),
                    epochs=20,
                    validation_data=validation_data.batch(1024),
                    verbose=1)

Epoch 1/20
13/13 [==============================] - 4s 331ms/step - loss: 0.8210 - accuracy: 0.5162 - val_loss: 0.6691 - val_accuracy: 0.5933
Epoch 2/20
13/13 [==============================] - 4s 271ms/step - loss: 0.6578 - accuracy: 0.6113 - val_loss: 0.6582 - val_accuracy: 0.6308
Epoch 3/20
13/13 [==============================] - 3s 262ms/step - loss: 0.6446 - accuracy: 0.6340 - val_loss: 0.6293 - val_accuracy: 0.6507
Epoch 4/20
13/13 [==============================] - 3s 268ms/step - loss: 0.6204 - accuracy: 0.6552 - val_loss: 0.6116 - val_accuracy: 0.6621
Epoch 5/20
13/13 [==============================] - 4s 283ms/step - loss: 0.6024 - accuracy: 0.6708 - val_loss: 0.5927 - val_accuracy: 0.6828
Epoch 6/20
13/13 [==============================] - 4s 279ms/step - loss: 0.5796 - accuracy: 0.6906 - val_loss: 0.5687 - val_accuracy: 0.7040
Epoch 7/20
13/13 [==============================] - 3s 267ms/step - loss: 0.5546 - accuracy: 0.7180 - val_loss: 0.5470 - val_accuracy: 0.7296
Epoch 

In [49]:
results = model.evaluate(raw_test.batch(1024), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.317
accuracy: 0.867
